[View in Colaboratory](https://colab.research.google.com/github/ahracho/TIL/blob/master/Fluent_Python/4_Text_Byte.ipynb)

## 4. 텍스트와 바이트

파이썬3부터는 인간이 사용하는 텍스트 문자열과 원시 바이트 시퀀스를 엄격히 구분하기 시작했다. 

### 4.1. 문자 문제

문자열은 '문자'의 열이라고 정의할 수 있는데, 여기서 '문자'가 무엇인지에 따라 데이터형이 달라진다. 파이썬 3 str에서 가져오는 항목은 유니코드 문자이다. 유니코드 표준에서는 문자의 단위 원소와 특정 바이트 표현을 명확히 구분한다. 
- 문자의 단위 원소(코드 포인트) : U+ 접두사를 붙여 4자리~6자리로 표현된 숫자. A라는 문자는 코드 포인트 U+0041로 표현된다. 
- 문자를 표현하는 실제 바이트는 사용하는 인코딩에 따라 달라진다. 인코딩은 코드 포인트를 바이트 시퀀스로 변한하는 알고리즘으로, UTF-8, UTF-16LE 등이 있다. 

In [2]:
s = "café"
print(len(s))
b = s.encode('utf8') # UTF-8 인코딩을 이용해서 str을 bytes로 인코딩한다.
print(b) # b'caf\xc3\xa9'
print(len(b))
print(b.decode('utf8'))

4
b'caf\xc3\xa9'
5
café


### 4.2. 바이트에 대한 기본 지식

파이썬 3에서 이진 시퀀스를 위해 사용되는 내장 자료형은 bytes와 bytearray 두가지가 있다. bytes는 불변형이고, bytearray는 가변형이다. bytes와 bytearray에 들어 있는 각 항목은 0~255 사이의 정수이다. 이진 시퀀스를 슬라이싱하면 언제나 동일한 자료형의 이진 시퀀스가 만들어지며, 슬라이스 길이가 1일 때도 마찬가지이다.

In [4]:
cafe = bytes("café", encoding = 'utf8')
print(cafe) # b'caf\xc3\xa9'

print(cafe[0])
print(cafe[:1]) # b'c' -> bytes를 슬라이싱해도 bytes이다

cafe_array = bytearray(cafe)
print(cafe_array)
print(cafe_array[-1:])

# s[0] == s[:1]인 시퀀스형은 str이 유일하다. 

b'caf\xc3\xa9'
99
b'c'
bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')


각 바이트 값은 다음과 같이 세가지 형태로 출력된다
- 화면에 출력 가능한 아스키 문자는 문자 그대로 출력된다.
- 탭, 개행문자, 캐리지 리턴, 백슬래시는 이스케이프 시퀀스로 출력한다.
- 그 외의 값은 16진수 이스케이프 시퀀스로 출력된다.

bytes와 bytearray는 format(), format_map() 메서드를 제외하고는 str이 제공하는 메서드를 모두 사용할 수 있고, 유니코드 데이터에 관련된 메서드도 지원한다. str 대신 이진 시퀀스로 정규표현식을 컴파일하면 정규 표현식 함수를 잉진 시퀀스에도 적용할 수 있다. 

In [8]:
print(bytes.fromhex("31 4B CE A9")) # 공백으로 구분된 16진수 쌍을 파싱해서 이진 시퀀스로 만든다.
print(bytes.fromhex("31 4B CE A9").decode())

b'1K\xce\xa9'
1KΩ


In [9]:
import array

numbers = array.array('h', [-2, -1, 0, 1, 2]) # short 타입 정수 5개
octets = bytes(numbers)
print(octets)

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'


#### 4.2.1. 구조체와 메모리 뷰

struct 모듈은 패킹된 바이트를 다양한 형의 필드로 구성된 튜플로 분석하고, 튜플을 패킹된 바이트로 변환하는 함수를 제공한다. 

In [0]:
import struct
fmt = "<3s3sHH" # <는 리틀엔디언, 3s3s는 3바이트 시퀀스 2개, HH는 16비트 정수 두개
with open('filter.gif', 'rb') as fp:
  img = memoryview(fp.read())
  
header = img[:10]
print(bytes(header))
print(struct.unpack(fmt, header)) # (종류, 버전, 너비, 높이) 튜플로 언패킹

del header
del img

### 4.3. 기본 인코더/디코더

In [10]:
for codec in ['latin_1', 'utf_8', 'utf_16'] :
  print(codec, "El Niño".encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


### 4.4. 인코딩/디코딩 문제 이해하기
#### 4.4.1. UnicodeEncodeError 처리하기


In [12]:
city = "São Paulo"
print(city.encode('utf_8'))
print(city.encode('utf_16'))
print(city.encode('iso8859_1'))
# print(city.encode('cp437')) # error 발생

print(city.encode('cp437', errors='ignore'))
print(city.encode('cp437', errors='replace')) # 인코딩할 수 없는 문자를 물음표로 치환
print(city.encode('cp437', errors='xmlcharrefreplace')) # XML 개체로 치환

b'S\xc3\xa3o Paulo'
b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
b'S\xe3o Paulo'
b'So Paulo'
b'S?o Paulo'
b'S&#227;o Paulo'


#### 4.4.2. UnicodeDecodeError 처리하기

이진 시퀀스를 텍스트로 변환할 때 정당한 문자로 변환할 수 없으면 UnicodeDecodeError가 발생한다. 하지만 많은 레거시 8비트 코덱은 무작위 비트 배열에 대해서도 에러를 발생시키지 않고 바이트 스트림으로 디코딩하기 때문에 잘못된 8비트 코덱을 사용하면 쓰레기 문자를 조용히 디코딩하게 된다. 

In [14]:
octets = b'Montr\xe9al'
print(octets.decode('cp1252'))
print(octets.decode('iso8859_7'))
print(octets.decode('koi8_r'))
# print(octets.decode('utf_8')) # error 발생
print(octets.decode('utf_8', errors='replace'))


Montréal
Montrιal
MontrИal
Montr�al


#### 4.4.3. 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError

파이썬 3에서는 UTF-8을 소스 코드 기본 인코딩 방식으로 사용하기 때문에 인코딩 선언 없이 비UTF-8로 인코딩된 .py 모듈을 로딩하면 에러 메시지가 발생한다. 그럴 땐 아래와 같이 소스 코드가 어떤 코덱을 사용하는지 명시해주면 된다.


In [15]:
# coding: cp1252

print('Ol�, Mundo!')

Ol�, Mundo!


#### 4.4.4. 바이트 시퀀스의 인코딩 방식을 알아내는 방식

Chardet 패키지는 바이트 스트림의 패턴을 분석하여 약 30가지 인코딩 방식을 알아낸다.

#### 4.4.5. BOM : 유용한 깨진 문자

In [18]:
u16 = "El Niño".encode("utf_16")
print(u16)
print(list(u16))

u8 = "El Niño".encode("utf_8")
print(u8)
print(list(u8))

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'
[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
b'El Ni\xc3\xb1o'
[69, 108, 32, 78, 105, 195, 177, 111]


"\xff\xfe"는 바이트 순서 표시(BOM)으로 리틀엔디언 바이트 순서를 나타낸다. U+0045(십진수 69)인 'E'는 69와 0으로 인코딩 되었다. 

In [19]:
u16le = "El Niño".encode("utf_16le") # 리틀 엔디언
print(list(u16le))

u16be = "El Niño".encode("utf_16be") # 빅 엔디언
print(list(u16be))

# 엔디언을 인코딩할 때 명시하였기 때문에 BOM은 나타나지 않는다.

[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]


### 4.5. 텍스트 파일 다루기

텍스트를 처리할 때는 입력시에 bytes를 str으로 디코딩하고, 프로그램 처리에서 텍스트만 처리하고, 출력할 때 바이트로 인코딩해서 사용한다. 보통은 open() 함수에서 인코딩/디코딩 작업을 수행하기 때문에 read()에서 str 객체를 가져오고 write()로 저장하면 된다. 

In [0]:
open('cafe.txt', 'w', encoding='utf_8').write('café') # utf-8로 인코딩했는데
open('cafe.txt').read() # 부를 땐 지정하지 않아 시스템 기본 인코딩 (Windows 1252)를 사용

In [0]:
fp = open('cafe.txt', 'w', encoding='utf_8')
fp.write("café") # 4 바이트 write
fp.close()


import os
print(os.stat('cafe.txt').st_size) # 5바이트 

fp2 = open('cafe.txt') # encoding = cp1252
print(fp2.read()) # e가 다른 글자로

fp3 = open('cafe.txt', encoding='utf_8')
print(fp3.read())

fp4 = open('cafe.txt', 'rb') # 이진 모드로 열기
print(fp4.read()) # bytes가 반환



#### 4.5.1. 기본 인코딩 설정

In [0]:
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

# 지역/시스템에 따라서 설정값이 다름

### 4.6. 제대로 비교하기 위해 유니코드 정규화하기



In [20]:
s1 = 'café'
s2 = 'cafe\u0301' # e + 발음기호
print(s1, s2) # 출력 모양은 같지만

print(len(s1), len(s2)) # 길이가 다름

print(s1 == s2)

café café
4 5
False


In [22]:
from unicodedata import normalize

s1 = 'café'
s2 = 'cafe\u0301' # e + 발음기호
print(len(s1), len(s2)) # 길이가 다름

print(len(normalize("NFC", s1)), len(normalize("NFC", s2)))
print(len(normalize("NFD", s1)), len(normalize("NFD", s2)))

print(normalize("NFC", s1) == normalize("NFC", s2))
print(normalize("NFD", s1) == normalize("NFD", s2))

4 5
4 4
5 5
True
True


In [24]:
from unicodedata import normalize, name

ohm = '\u2126'
print(name(ohm))

ohm_c = normalize("NFC", ohm)
print(name(ohm_c))

print(ohm == ohm_c)
print(normalize("NFC", ohm) == normalize("NFC", ohm_c)) # NFC를 쓰면 사실 다른 건데 같다고 정규화

OHM SIGN
GREEK CAPITAL LETTER OMEGA
False
True


In [30]:
from unicodedata import normalize, name

half = "½"
print(normalize("NFKC", half))

four_squared = '42'
print(normalize("NFKC", four_squared))

micro = 'µ'
micro_kc = normalize("NFKC", micro)
print(micro, micro_kc)
print(ord(micro), ord(micro_kc))
print(name(micro), name(micro_kc)) # NFKC는 구분

1⁄2
42
µ μ
181 956
MICRO SIGN GREEK SMALL LETTER MU


#### 4.6.1. 케이스 폴딩

모든 텍스트를 소문자로 변환하는 연산

In [33]:
micro = 'µ'
print(name(micro))
micro_cf = micro.casefold()
print(name(micro_kc))

print(micro, micro_cf)

eszett = 'ß'
print(name(eszett))
eszett_cf = eszett.casefold()
# print(name(eszett_cf)) # error
print(eszett, eszett_cf)

MICRO SIGN
GREEK SMALL LETTER MU
µ μ
LATIN SMALL LETTER SHARP S
ß ss


#### 4.6.2. 정규화된 텍스트 매칭을 위한 유틸리티 함수

NFC, NFD는 안전하며 유니코드 문자열을 적절히 비교할 수 있게 해준다. NFC는 대부분의 어플리케이션에서 사용할 수 있는 최고의 정규화 형태며, str.casefold()는 대소문자 구분 없이 문자를 비교할 때 가장 좋은 방법이다. 



In [0]:
"""
Utility functions for normalized Unicode string comparison.

Using Normal Form C, case sensitive:

    >>> s1 = 'café'
    >>> s2 = 'cafe\u0301'
    >>> s1 == s2
    False
    >>> nfc_equal(s1, s2)
    True
    >>> nfc_equal('A', 'a')
    False

Using Normal Form C with case folding:

    >>> s3 = 'Straße'
    >>> s4 = 'strasse'
    >>> s3 == s4
    False
    >>> nfc_equal(s3, s4)
    False
    >>> fold_equal(s3, s4)
    True
    >>> fold_equal(s1, s2)
    True
    >>> fold_equal('A', 'a')
    True

"""

from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() ==
            normalize('NFC', str2).casefold())


#### 4.6.3. 극단적인 '정규화' : 발음 구별 기호 제거하기



In [35]:
# BEGIN SHAVE_MARKS
import unicodedata
import string


def shave_marks(txt):
    """Remove all diacritic marks"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))  # <2>
    return unicodedata.normalize('NFC', shaved)  # <3>
# END SHAVE_MARKS

order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”' # è, ç, í 대체
print(shave_marks(order))
greek = 'Ζέφυρος, Zéfiro' # έ, é 대체
print(shave_marks(greek))

“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”
Ζεφυρος, Zefiro


In [37]:
# BEGIN SHAVE_MARKS_LATIN
def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # <2>
            continue  # ignore diacritic on Latin base char
        keepers.append(c)                             # <3>
        # if it isn't combining char, it's a new base char
        if not unicodedata.combining(c):              # <4>
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)   # <5>
# END SHAVE_MARKS_LATIN


print(shave_marks_latin(order))
print(shave_marks_latin(greek))

“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”
Ζέφυρος, Zefiro


In [38]:
# BEGIN ASCIIZE
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  # <1>
                                               """'f"*^<''""---~>""")

multi_map = str.maketrans({  # <2>
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  # <3>


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  # <4>


# 서양 텍스트에서 널리 사용되는 기호들을 아스키에 해당하는 문자로 바꾸는
def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))     # <5>
    no_marks = no_marks.replace('ß', 'ss')          # <6>
    return unicodedata.normalize('NFKC', no_marks)  # <7>
# END ASCIIZE


print(dewinize(order))
print(asciize(order))

print(dewinize(greek))
print(asciize(greek))

"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."
"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."
Ζέφυρος, Zéfiro
Ζέφυρος, Zefiro


### 4.7. 유니코드 텍스트 정렬하기

문자열의 경우에는 각 단어의 코드 포인트를 비교하여 정렬하는데, 비아스키 문자를 사용하는 경우에는 부적절한 결과가 발생할 수 있다. 라틴 알파벳을 사용하는 언어에서는 정렬할 때 악센트와 갈고리형 기호가 영향을 미치지 않는데, 정렬 시에 잘못되는 경우가 발생한다. 

파이썬에서 비아스키 텍스트는 locale.strxfrm() 함수를 사용하여 변환하는 것이 표준이고, 이를 사용하기 전에는 setlocale(LC_COLLATE, 지역언어)를 호출해야 한다. 

### 4.8. 유니코드 데이터베이스

유니코드 표준은 수많은 구조화된 텍스트 파일의 형태로 하나의 완전한 데이터베이스를 제공한다. 데이터베이스에는 코드 포인트를 문자명과 매칭한 테이블 뿐만 아니라 문자를 출력할 수 있는지, 문자인지/숫자인지/수치형 기호인지 등의 정보를 담고 있다. 

In [39]:
# BEGIN NUMERICS_DEMO
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),                       # <1>
          char.center(6),                             # <2>
          're_dig' if re_digit.match(char) else '-',  # <3>
          'isdig' if char.isdigit() else '-',         # <4>
          'isnum' if char.isnumeric() else '-',       # <5>
          format(unicodedata.numeric(char), '5.2f'),  # <6>
          unicodedata.name(char),                     # <7>
          sep='\t')
# END NUMERICS_DEMO


U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


### 4.9. 이중 모드 str 및 bytes API

표준 라이브러리에는 str이나 bytes 인수를 모두 받으며, 인수의 자료형에 따라 다르게 작동하는 함수들이 있는데, re와 os가 대표적이다. 

#### 4.9.1. 정규표현식에서의 str과 bytes

bytes로 정규표현식을 만들면 \d와 \w 같은 패턴은 아스키 문자만 매칭되지만, str로 이 패턴을 만들면 유니코드 숫자나 문자도 매칭된다. 



In [40]:
# BEGIN RE_DEMO
import re

re_numbers_str = re.compile(r'\d+')     # <1> str 타입
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')  # <2> bytes 타입
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"  # <3>
            " as 1729 = 1³ + 12³ = 9³ + 10³.")        # <4>

text_bytes = text_str.encode('utf_8')  # <5>

print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))      # <6>
print('  bytes:', re_numbers_bytes.findall(text_bytes))  # <7>
print('Words')
print('  str  :', re_words_str.findall(text_str))        # <8>
print('  bytes:', re_words_bytes.findall(text_bytes))    # <9>
# END RE_DEMO


Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


#### 4.9.2. os  모듈 함수에서 str과 bytes

GNU/리눅스 커널은 유니코드를 모른다. 그렇기 때문에 os의 파일명은 어떠한 인코딩 체계에서도 올바르지 않은 바이트 시퀀스로 구성되어 있으며 str으로 디코딩할 수 없다. 이 문제를 해결하기 위해서 파일명이나 경로명을 받는 모든 os 함수는 str이나 bytes 형의 인수를 받는다. str으로 호출하면 sys.getfilesystsemencoding() 함수에 의해 지정된 코덱을 이용해서 자동으로 변환되고 동일 코덱을 이용해서 디코딩 된다. 